Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [14]:
train_subset = 10000
beta = 0.05
graph = tf.Graph()
with graph.as_default():
    tf_train_X = tf.constant(train_dataset[:train_subset, :])
    tf_train_y = tf.constant(train_labels[:train_subset, :])
    tf_valid_X = tf.constant(valid_dataset)
    tf_valid_y = tf.constant(valid_labels)
    tf_test_X = tf.constant(test_dataset)
    print(tf_train_X.get_shape())
    W = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b = tf.Variable(tf.zeros([num_labels]))
    print(W.get_shape())
    logits = tf.matmul(tf_train_X, W) + b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_y))
    loss = loss + 0.5*beta*tf.nn.l2_loss(W)
    
    op = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_predict = tf.nn.softmax(logits)
    valid_predict = tf.nn.softmax(tf.matmul(tf_valid_X, W) + b)
    test_predict = tf.nn.softmax(tf.matmul(tf_test_X, W) + b)
    

TensorShape([Dimension(10000), Dimension(784)])
TensorShape([Dimension(784), Dimension(10)])


In [18]:
num_steps = 801

def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in xrange(num_steps):
        _, l, predict = session.run([op, loss, train_predict])
        
        if step % 100 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predict, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_predict.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_predict.eval(), test_labels))

Loss at step 0: 93.403900
Training accuracy: 10.9%
Validation accuracy: 14.7%
Loss at step 100: 6.582597
Training accuracy: 77.8%
Validation accuracy: 76.9%
Loss at step 200: 1.178381
Training accuracy: 83.4%
Validation accuracy: 81.7%
Loss at step 300: 0.780907
Training accuracy: 83.8%
Validation accuracy: 82.1%
Loss at step 400: 0.750403
Training accuracy: 83.9%
Validation accuracy: 82.1%
Loss at step 500: 0.747816
Training accuracy: 83.9%
Validation accuracy: 82.1%
Loss at step 600: 0.747487
Training accuracy: 83.9%
Validation accuracy: 82.1%
Loss at step 700: 0.747391
Training accuracy: 84.0%
Validation accuracy: 82.1%
Loss at step 800: 0.747344
Training accuracy: 84.0%
Validation accuracy: 82.1%
Test accuracy: 88.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
batch_size = 128
beta = 0.05
graph = tf.Graph()
with graph.as_default():
    tf_train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_X = tf.constant(valid_dataset)
    tf_valid_y = tf.constant(valid_labels)
    tf_test_X = tf.constant(test_dataset)
    print(tf_train_X.get_shape())
    W = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b = tf.Variable(tf.zeros([num_labels]))
    print(W.get_shape())
    logits = tf.matmul(tf_train_X, W) + b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_y))
    loss = loss + 0.5*beta*tf.nn.l2_loss(W)
    
    op = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_predict = tf.nn.softmax(logits)
    valid_predict = tf.nn.softmax(tf.matmul(tf_valid_X, W) + b)
    test_predict = tf.nn.softmax(tf.matmul(tf_test_X, W) + b)
    

TensorShape([Dimension(128), Dimension(784)])
TensorShape([Dimension(784), Dimension(10)])


In [25]:
num_steps = 801

def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {
            tf_train_X: batch_data,
            tf_train_y: batch_labels
        }
        _, l, predict = session.run([op, loss, train_predict], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_predict.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_predict.eval(), test_labels))

Minibatch loss at step 0: 96.287888
Minibatch accuracy: 5.5%
Validation accuracy: 8.9%
Minibatch loss at step 500: 0.942569
Minibatch accuracy: 78.1%
Validation accuracy: 80.6%
Test accuracy: 86.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [58]:

batch_size = 128
beta = 0.0005
hidden_nodes = 1024
graph = tf.Graph()
with graph.as_default():
    tf_train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_X = tf.constant(valid_dataset)
    tf_valid_y = tf.constant(valid_labels)
    tf_test_X = tf.constant(test_dataset)

    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=1.0/float(image_size)))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    h1 = tf.nn.relu(tf.matmul(tf_train_X, W1) + b1)
    keep_rate = tf.placeholder("float")
    h1 = tf.nn.dropout(h1, keep_rate)
    
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(h1, W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_y))
    loss = loss + 0.5*beta*tf.nn.l2_loss(W)
    
    op = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_predict = tf.nn.softmax(logits)
    valid_predict = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_X, W1) + b1), W2) + b2)
    test_predict = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_X, W1) + b1), W2) + b2)
    

In [59]:
num_steps = 5001

def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {
            tf_train_X: batch_data,
            tf_train_y: batch_labels,
            keep_rate: 0.5
        }
        _, l, predict = session.run([op, loss, train_predict], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_predict.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_predict.eval(), test_labels))

Minibatch loss at step 0: 17.263939
Minibatch accuracy: 9.4%
Validation accuracy: 36.1%
Minibatch loss at step 500: 7.267199
Minibatch accuracy: 73.4%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 5.732685
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 2.616941
Minibatch accuracy: 78.1%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 1.791963
Minibatch accuracy: 85.9%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 1.595532
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 1.372820
Minibatch accuracy: 82.8%
Validation accuracy: 86.3%
Minibatch loss at step 3500: 1.185725
Minibatch accuracy: 88.3%
Validation accuracy: 86.6%
Minibatch loss at step 4000: 1.204726
Minibatch accuracy: 85.2%
Validation accuracy: 87.1%
Minibatch loss at step 4500: 1.278616
Minibatch accuracy: 83.6%
Validation accuracy: 86.8%
Minibatch loss at step 5000: 1.328139
Minibatch accuracy: 85.9%
Validation accuracy: 87.6%
Tes

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [87]:

batch_size = 128
beta = 0.0005
h1_nodes = 1024
h2_nodes = 512
h3_nodes = 256
h4_nodes = 128
decay_steps = 10000
decay_rate = 0.96

graph = tf.Graph()
with graph.as_default():
    tf_train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_X = tf.constant(valid_dataset)
    tf_valid_y = tf.constant(valid_labels)
    tf_test_X = tf.constant(test_dataset)
    keep_rate = tf.placeholder("float")
    global_step = tf.Variable(0)
    
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, h1_nodes], stddev=1.0/float(image_size)))
    b1 = tf.Variable(tf.zeros([h1_nodes]))
    h1 = tf.nn.relu(tf.matmul(tf_train_X, W1) + b1)
    h1 = tf.nn.dropout(h1, keep_rate)

    W2 = tf.Variable(tf.truncated_normal([h1_nodes, h2_nodes], stddev=1.0/float(h2_nodes)))
    b2 = tf.Variable(tf.zeros([h2_nodes]))
    h2 = tf.nn.relu(tf.matmul(h1, W2) + b2)
    h2 = tf.nn.dropout(h2, keep_rate)
    
    W3 = tf.Variable(tf.truncated_normal([h2_nodes, h3_nodes], stddev=1.0/float(h3_nodes)))
    b3 = tf.Variable(tf.zeros([h3_nodes]))
    h3 = tf.nn.relu(tf.matmul(h2, W3) + b3)
    h3 = tf.nn.dropout(h3, keep_rate)
    
    W4 = tf.Variable(tf.truncated_normal([h3_nodes, num_labels], stddev=1.0/float(num_labels)))
    b4 = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(h3, W4) + b4
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_y))
    loss = loss + 0.5*beta*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4))

    learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    train_predict = tf.nn.softmax(logits)
    valid_predict = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_X, W1) + b1), W2) + b2), W3) + b3), W4) + b4)
    test_predict = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_X, W1) + b1), W2) + b2), W3) + b3), W4) + b4)
    

In [88]:
num_steps = 10001

def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {
            tf_train_X: batch_data,
            tf_train_y: batch_labels,
            keep_rate: 0.8
        }
        _, l, predict = session.run([op, loss, train_predict], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predict, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_predict.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_predict.eval(), test_labels))

Minibatch loss at step 0: 2.404773
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.639067
Minibatch accuracy: 83.6%
Validation accuracy: 85.1%
Minibatch loss at step 1000: 0.626695
Minibatch accuracy: 80.5%
Validation accuracy: 86.8%
Minibatch loss at step 1500: 0.496844
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatch loss at step 2000: 0.593494
Minibatch accuracy: 84.4%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 0.452017
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 3000: 0.559841
Minibatch accuracy: 85.9%
Validation accuracy: 88.5%
Minibatch loss at step 3500: 0.470520
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 4000: 0.413239
Minibatch accuracy: 92.2%
Validation accuracy: 89.4%
Minibatch loss at step 4500: 0.374800
Minibatch accuracy: 89.8%
Validation accuracy: 89.5%
Minibatch loss at step 5000: 0.435176
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Min